In [45]:
# import json
# import geojson
# import pandas as pd
# import matplotlib.pyplot as plt
# import numpy as np
# from sklearn.linear_model import LinearRegression
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import mean_squared_error, r2_score

In [46]:
# with open('intermediate_file_paths.json') as output_path_file:
#     output_paths = json.load(output_path_file)

# SI_PER_FIRE_INPUT_PATH = output_paths.get('stage1_si_per_fire_csv')
# SCALED_SI_PER_YEAR_INPUT_PATH = output_paths.get('stage2_scaled_si_per_year_csv')

# # Load dataset
# yearly_smoke_impact_df = pd.read_csv(SCALED_SI_PER_YEAR_INPUT_PATH)
# si_per_fire_df = pd.read_csv(SI_PER_FIRE_INPUT_PATH)

In [47]:
# print(yearly_smoke_impact_df.columns)

In [48]:
# print(si_per_fire_df.columns)

In [68]:
# %% [markdown]
# ## Update to include info about calculation steps

# %%
import json
import geojson
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures

with open('intermediate_file_paths.json') as output_path_file:
    output_paths = json.load(output_path_file)

SI_PER_FIRE_INPUT_PATH = output_paths.get('stage1_si_per_fire_csv')
SCALED_SI_PER_YEAR_INPUT_PATH = output_paths.get('stage2_scaled_si_per_year_csv')

# Load dataset
yearly_smoke_impact_df = pd.read_csv(SCALED_SI_PER_YEAR_INPUT_PATH)
si_per_fire_df = pd.read_csv(SI_PER_FIRE_INPUT_PATH)

# Define weights for 'total_acres_burned' and 'distance'
weights = {'total_acres_burned': 1, 'distance': 1}

# Calculate the average distance per year
avg_dist_df = si_per_fire_df.groupby('fire_year').agg(
    avg_dist_from_city=('distance', 'mean')
).reset_index()

# Filter data to include only years after 1984
yearly_smoke_impact_filtered_df = yearly_smoke_impact_df[yearly_smoke_impact_df['year'] > 1984]
filtered_avg_dist_df = avg_dist_df[avg_dist_df['fire_year'] > 1984]

# Prepare data for prediction using original data (without moving average)
combined_df = yearly_smoke_impact_filtered_df[['scaled_avg_daily_smoke_impact', 'year', 'total_acres_burned']].merge(
    filtered_avg_dist_df[['fire_year', 'avg_dist_from_city']],
    left_on='year', right_on='fire_year', how='left'
)

# Apply weights to the relevant predictor variables
combined_df['total_acres_burned_weighted'] = combined_df['total_acres_burned'] * weights['total_acres_burned']
combined_df['distance_weighted'] = combined_df['avg_dist_from_city'] * weights['distance']

# Identify which years have NaN values
na_years = combined_df[combined_df.isna().any(axis=1)]['year'].unique()
print(f"Years with NaN values: {na_years}")

print(combined_df['scaled_avg_daily_smoke_impact'].max())

# Remove outliers using Interquartile Range (IQR) method
Q1 = combined_df['scaled_avg_daily_smoke_impact'].quantile(0.25)
Q3 = combined_df['scaled_avg_daily_smoke_impact'].quantile(0.75)

IQR = Q3 - Q1

print(f'Q1: {Q1}, Q3: {Q3}, IQR: {IQR}')
# combined_df = combined_df[(combined_df['scaled_avg_daily_smoke_impact'] >= (Q1 - 1.2 * IQR)) & 
#                           (combined_df['scaled_avg_daily_smoke_impact'] <= (Q3 + 1.2 * IQR))]

combined_df = combined_df[(combined_df['scaled_avg_daily_smoke_impact'] >= (Q1 - 0.5 * IQR)) & 
                          (combined_df['scaled_avg_daily_smoke_impact'] <= (Q3 + 0.5 * IQR))]

# Remove rows with NaN values in features or labels
combined_df = combined_df.dropna()

print(combined_df['scaled_avg_daily_smoke_impact'].max())

# Check if there are enough samples left after filtering
if combined_df.shape[0] == 0:
    raise ValueError("No samples left after filtering for outliers and NaN values. Consider relaxing the filtering criteria.")

# Normalize total acres burned and distance
combined_df['total_acres_burned_normalized'] = combined_df['total_acres_burned'] / combined_df['total_acres_burned'].max()
combined_df['distance_normalized'] = combined_df['avg_dist_from_city'] / 650

print(combined_df)

# Prepare the features and target variable
features = combined_df[['total_acres_burned_normalized', 'distance_normalized']]
labels = combined_df['scaled_avg_daily_smoke_impact'].values

# Add interaction terms using PolynomialFeatures
poly = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)
features_interaction = poly.fit_transform(features)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features_interaction, labels, test_size=0.2, random_state=42)

# Create and train the linear regression model with K-Fold Cross-Validation
model = LinearRegression()
kf = KFold(n_splits=5, shuffle=True, random_state=42)
cross_val_mse = cross_val_score(model, features_interaction, labels, cv=kf, scoring='neg_mean_squared_error')
cross_val_r2 = cross_val_score(model, features_interaction, labels, cv=kf, scoring='r2')

print(f"Cross-Validation Mean Squared Error (MSE): {-np.mean(cross_val_mse)}")
print(f"Cross-Validation R-squared (R2): {np.mean(cross_val_r2)}")





Years with NaN values: [2021]
52.782934279853016
Q1: 16.74033345730031, Q3: 34.60592853216895, IQR: 17.865595074868644
41.381605819145946
    scaled_avg_daily_smoke_impact  ...  distance_normalized
0                       13.223345  ...             0.687161
3                       22.046619  ...             0.616994
4                       13.755830  ...             0.603956
5                        8.653858  ...             0.534834
7                       10.522113  ...             0.588911
8                       13.372834  ...             0.622064
9                       24.890273  ...             0.605573
10                       9.156539  ...             0.424505
11                      41.381606  ...             0.554866
12                      13.160343  ...             0.614723
13                      16.820036  ...             0.618477
14                      16.501227  ...             0.606933
15                      27.040809  ...             0.585323
16                    